In [0]:
val filePath = "bdad/dataset/goodreads_reviews_dedup.json"

val reviews_df = spark.read.json(filePath)
z.show(reviews_df)

In [1]:
reviews_df.printSchema()

In [2]:
reviews_df.cache().count

### Data Cleaning

Dropping the columns that are not required for the analytics [date_added, date_updated]

In [5]:
val reviews_base_df = reviews_df.select(
    "book_id",
    "user_id",
    "review_id",
    "review_text",
    "rating",
    "n_votes",
    "n_comments",
    "read_at",
    "started_at",
    "date_updated"
)

z.show(reviews_base_df)

In [6]:
val non_null_reviews_df = reviews_base_df.where("book_id is not NULL").where("user_id is not NULL").where("review_id is not NULL")

z.show(non_null_reviews_df)

In [7]:
non_null_reviews_df.count

Setting default value for values for columns that are allowed to be empty

In [9]:

import org.apache.spark.sql.functions.{to_date, to_timestamp}

val casted_reviews_df = non_null_reviews_df.withColumnRenamed("date_updated","date_modified").withColumn("n_votes", col("n_votes").cast("long")).withColumn("n_comments", col("n_comments").cast("long")).withColumn("rating", col("rating").cast("int")).withColumn("read_at", to_timestamp($"read_at", "EEE MMM dd HH:mm:ss z yyyy")).withColumn("started_at", to_timestamp($"started_at", "EEE MMM dd HH:mm:ss z yyyy")).withColumn("date_modified", to_timestamp($"date_modified", "EEE MMM dd HH:mm:ss z yyyy"))

val cleaned_reviews_df = casted_reviews_df.na.fill(0, Array("n_votes", "n_comments", "rating"))

cleaned_reviews_df.printSchema()



Printing sample record after cleaning

In [11]:
z.show(cleaned_reviews_df)

### Data Profiling

#### Total number of reviews

In [14]:
cleaned_reviews_df.cache().count


#### Total books reviewed

In [16]:
cleaned_reviews_df.select("book_id").distinct().count()


#### Count of reviews given by each user

In [18]:

val user_count_df = cleaned_reviews_df.groupBy("user_id").count()

z.show(user_count_df)

#### Count of reviews given for each book

In [20]:
val book_count_df = cleaned_reviews_df.groupBy("book_id").count()

z.show(book_count_df)

#### Average rating of each book

In [22]:
val average_rating_df = cleaned_reviews_df.groupBy("book_id").agg(avg("rating"))

z.show(average_rating_df)

#### Book count for each rating

In [24]:
val rating_book_count_df = cleaned_reviews_df.groupBy("rating").count()

z.show(rating_book_count_df)

#### Average length of review text

In [26]:
cleaned_reviews_df.withColumn("review_text_length", length(col("review_text"))).select(avg($"review_text_length")).show()


In [27]:
cleaned_reviews_df.write.mode("overwrite").parquet("bdad/dataset/reviews.parquet")